In [9]:
class SudokuSolver:
    def __init__(self, board):
        self.board = board  
        self.size = 9
        self.domain = { (r, c): set(range(1, 10)) for r in range(self.size) for c in range(self.size) if board[r][c] == 0 }

    def is_valid(self, r, c, value):
        """Check if placing value at board[r][c] is valid."""
        for i in range(9):
            if self.board[r][i] == value or self.board[i][c] == value:
                return False

        block_row, block_col = 3 * (r // 3), 3 * (c // 3)
        for i in range(3):
            for j in range(3):
                if self.board[block_row + i][block_col + j] == value:
                    return False

        return True

    def forward_checking(self, r, c, value):
        """Prune the domain of neighbors by removing value."""
        affected = []
        for i in range(9):
            if (r, i) in self.domain and value in self.domain[(r, i)]:
                self.domain[(r, i)].remove(value)
                affected.append((r, i))

            if (i, c) in self.domain and value in self.domain[(i, c)]:
                self.domain[(i, c)].remove(value)
                affected.append((i, c))

        block_row, block_col = 3 * (r // 3), 3 * (c // 3)
        for i in range(3):
            for j in range(3):
                pos = (block_row + i, block_col + j)
                if pos in self.domain and value in self.domain[pos]:
                    self.domain[pos].remove(value)
                    affected.append(pos)

        return affected

    def restore_domain(self, affected, value):
        """Restore the domain after backtracking."""
        for pos in affected:
            self.domain[pos].add(value)

    def arc_consistency(self):
        """Ensure all variables are arc-consistent."""
        queue = list(self.domain.keys())
        while queue:
            r, c = queue.pop(0)
            for value in list(self.domain[(r, c)]):
                if not self.is_valid(r, c, value):
                    self.domain[(r, c)].remove(value)
                if len(self.domain[(r, c)]) == 0:
                    return False  # Failure
        return True

    def find_unassigned(self):
        """Find the next unassigned variable (MRV heuristic)."""
        unassigned = [var for var in self.domain if len(self.domain[var]) > 0]
        return min(unassigned, key=lambda var: len(self.domain[var]), default=None)

    def backtrack(self):
        """Main backtracking function."""
        pos = self.find_unassigned()
        if not pos:
            return True

        r, c = pos
        for value in self.domain[pos]:
            if self.is_valid(r, c, value):
                self.board[r][c] = value
                affected = self.forward_checking(r, c, value)

                if self.arc_consistency():
                    if self.backtrack():
                        return True

                self.restore_domain(affected, value)
                self.board[r][c] = 0

        return False

    def solve(self):
        if not self.arc_consistency():
            return False
        return self.backtrack()

    def print_board(self):
        for row in self.board:
            print(' '.join(str(num) if num != 0 else '.' for num in row))

# Example Usage
sudoku_board = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

solver = SudokuSolver(sudoku_board)
if solver.solve():
    solver.print_board()
else:
    print("No solution exists.")


5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9
